In [1]:
# utils
import pandas as pd
import os
import numpy as np
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import ast
import re
from scipy.stats import ttest_rel

# evaluation 
from evaluate import load

In [2]:
# open files in generations directory
def open_files(directory):
    files = os.listdir(directory)
    files.sort()
    return files

In [3]:
models_generations = {
    'en': {},
    'it': {},
    'ru': {},
    'ge': {}
}

files = open_files('generations')
for file in files:
    model = re.sub(r"^fine-tuned-|-exp2-gen\d+.csv", "", file)
    gen = re.search(r"gen\d+", file).group()

    model_generations = pd.read_csv('generations/' + file)
    # convert string to list
    model_generations['actual'] = model_generations['actual'].apply(ast.literal_eval)
    
    # if containt -en-
    if '-en-' in file:
        if model not in models_generations['en']:
            models_generations['en'][model] = {}
        models_generations['en'][model][gen] = model_generations
        print(f"model: {model} - gen: {gen}, lang: en")
    elif '-it-' in file:
        if model not in models_generations['it']:
            models_generations['it'][model] = {}
        models_generations['it'][model][gen] = model_generations
        print(f"model: {model} - gen: {gen}, lang: it")
    elif '-ge-' in file:
        if model not in models_generations['ge']:
            models_generations['ge'][model] = {}
        models_generations['ge'][model][gen] = model_generations
        print(f"model: {model} - ge: {gen}, lang: ge")
    elif '-ru-' in file:
        if model not in models_generations['ru']:
            models_generations['ru'][model] = {}
        models_generations['ru'][model][gen] = model_generations
        print(f"model: {model} - gen: {gen}, lang: ru")

model: Llama-3.1-8B-Instruct-en - gen: gen0, lang: en
model: Llama-3.1-8B-Instruct-en - gen: gen1, lang: en
model: Llama-3.1-8B-Instruct-en - gen: gen2, lang: en
model: Llama-3.1-8B-Instruct-ge - ge: gen0, lang: ge
model: Llama-3.1-8B-Instruct-ge - ge: gen1, lang: ge
model: Llama-3.1-8B-Instruct-ge - ge: gen2, lang: ge
model: Llama-3.1-8B-Instruct-it - gen: gen0, lang: it
model: Llama-3.1-8B-Instruct-it - gen: gen1, lang: it
model: Llama-3.1-8B-Instruct-it - gen: gen2, lang: it
model: Llama-3.1-8B-Instruct-ru - gen: gen0, lang: ru
model: Llama-3.1-8B-Instruct-ru - gen: gen1, lang: ru
model: Llama-3.1-8B-Instruct-ru - gen: gen2, lang: ru
model: Mistral-Nemo-Instruct-2407-en - gen: gen0, lang: en
model: Mistral-Nemo-Instruct-2407-en - gen: gen1, lang: en
model: Mistral-Nemo-Instruct-2407-en - gen: gen2, lang: en
model: Mistral-Nemo-Instruct-2407-ge - ge: gen0, lang: ge
model: Mistral-Nemo-Instruct-2407-ge - ge: gen1, lang: ge
model: Mistral-Nemo-Instruct-2407-ge - ge: gen2, lang: ge
mode

## Valutazione automatica

### <a href="https://huggingface.co/spaces/evaluate-metric/bleu">Bleu</a>

In [4]:
bleu = load("bleu")

Using the latest cached version of the module from C:\Users\OliverioM\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--bleu\9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Wed Sep  4 17:21:14 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.


In [5]:
bleu_scores = {'en': {}, 'it': {}, 'ge': {}, 'ru': {}}

for lang in models_generations:
    for model in models_generations[lang]:
        model_bleu_scores = []
        print(f'Language: {lang}, Model: {model}')
        
        for gen in sorted(models_generations[lang][model]):  # Ensure order (gen0, gen1, gen2)
            references = models_generations[lang][model][gen]['actual']
            predictions = models_generations[lang][model][gen]['prediction']

            gen_bleu_scores = []
            for i in tqdm(range(len(references)), desc="Calcolo punteggi BLEU"):
                results = bleu.compute(predictions=[predictions[i]], references=[references[i]])
                gen_bleu_scores.append(results["bleu"])

            model_bleu_scores.append(gen_bleu_scores)  # Store BLEU for each generation
        
        # Store the three BLEU scores instead of their mean
        bleu_scores[lang][model] = model_bleu_scores
        print(f'BLEU scores for {model} ({lang}): {model_bleu_scores}')
        print(f'Average: {np.mean(model_bleu_scores)}')
        print()

Language: en, Model: Llama-3.1-8B-Instruct-en


Calcolo punteggi BLEU: 100%|██████████| 383/383 [00:03<00:00, 124.96it/s]


BLEU scores for Llama-3.1-8B-Instruct-en (en): [[1.0, 0.6389431042462724, 0.0, 0.5431501067716656, 0.16245284024690473, 0.6909485625039599, 0.45986108016994914, 0.4554383163361387, 1.0, 0.7748677442328399, 0.45444956236094664, 0.604349704366814, 1.0, 1.0, 0.8891397050194614, 1.0, 0.8857000285382948, 1.0, 0.7028316885452014, 0.9621954581957615, 0.6524729016168254, 0.6836460043639834, 0.2975928234249097, 1.0, 0.94149097734812, 0.537284965911771, 1.0, 0.5691068337963667, 0.5883105707205901, 0.4408231875586727, 0.8307018474412792, 0.43550229045828076, 0.8225681733052571, 0.7039070022305045, 0.789357525105527, 1.0, 0.6909485625039599, 0.5671276403520978, 1.0, 0.8266188889979827, 0.4839281708164425, 0.48633831680799433, 0.40096084846312413, 0.5891223348606782, 0.788200954197212, 0.5348765896724532, 0.7186082239261685, 1.0, 1.0, 0.5757575636202255, 1.0, 0.8768419153065673, 1.0, 0.317023313852343, 0.640751803175311, 0.7513763122938849, 1.0, 0.8907003299890247, 0.7653312225290797, 0.70168793912

Calcolo punteggi BLEU: 100%|██████████| 383/383 [00:03<00:00, 116.23it/s]


BLEU scores for Mistral-Nemo-Instruct-2407-en (en): [[1.0, 0.6389431042462724, 0.0, 0.5671687320916967, 0.12064803452402914, 0.8870909062679921, 0.37283992459416565, 0.35406256950112003, 1.0, 0.7748677442328399, 0.4287232854924631, 0.7003652415618387, 1.0, 0.0, 0.8891397050194614, 1.0, 0.8857000285382948, 1.0, 0.0, 1.0, 0.5005335865721833, 0.6903822463805598, 0.7311104457090247, 0.8648454150926429, 0.605540583950642, 0.537284965911771, 1.0, 0.6680785126715956, 0.5165213935736707, 0.4408231875586727, 0.5814307369682193, 0.39242591746953154, 0.8080952687355902, 0.5233694611909245, 0.789357525105527, 0.6238986072117501, 0.7184162110235452, 0.9584146563694087, 0.7985065516266613, 0.6767045368190813, 0.37704209027116137, 0.48633831680799433, 0.2562956311149168, 0.5927478741786792, 0.5654397417534262, 0.5737014600396702, 0.7186082239261685, 0.9234732618882052, 1.0, 0.5757575636202255, 1.0, 0.8768419153065673, 1.0, 0.317023313852343, 0.640751803175311, 0.7361410881038141, 1.0, 0.7557847762798

...

BLEU scores for Llama-3.1-8B-Instruct-ge (ge): [[1.0, 0.8948393168143697, 0.0, 0.4562367262019882, 0.0, 0.6348073328850725, 0.46330597318302275, 0.5275980999336181, 0.0, 1.0, 0.4016650474304947, 0.9071112036324318, 0.7016879391277371, 1.0, 0.7476743906106103, 0.4548019047027907, 0.5757575636202255, 1.0, 0.6341922683775969, 0.7476743906106103, 0.7192367996522091, 0.7997605032399495, 0.5312583871630396, 0.541822042580596, 0.7487402156832422, 1.0, 1.0, 0.0, 0.5192108761155978, 0.0, 0.3549481056010053, 0.6623645504445912, 0.8219655850102567, 0.5373698475633983, 0.4527471870952893, 1.0, 0.43317377303208376, 0.8743414417652072, 1.0, 0.8235296495684443, 0.2551219280636033, 0.669961753544709, 0.196046355324564, 0.0, 0.6803749333171202, 0.6382401024752941, 0.7825422900366437, 0.8265168183793802, 1.0, 0.5747078645171895, 1.0, 0.8696398662122882, 1.0, 0.0, 0.5038601987543927, 0.5154337627145953, 0.8931539818068694, 0.7362096820824198, 0.6267671821810656, 0.7016879391277371, 0.0, 0.700961800242092

Calcolo punteggi BLEU: 100%|██████████| 383/383 [00:03<00:00, 110.85it/s]


BLEU scores for Mistral-Nemo-Instruct-2407-ge (ge): [[1.0, 0.8948393168143697, 0.0, 0.4562367262019882, 0.0, 0.4873483706735273, 0.44130156394078607, 0.5030760719034105, 0.0, 1.0, 0.3672654922979082, 0.6811873227441255, 0.537284965911771, 0.0, 0.7476743906106103, 0.4548019047027907, 0.5757575636202255, 0.5623413251903491, 0.6598573198708221, 1.0, 0.5935409122738204, 0.6998534451614725, 0.37239098949398236, 0.5773502691896258, 0.45180100180492244, 0.537284965911771, 1.0, 0.2451829991731284, 0.47238380914874, 0.4408231875586727, 0.3549481056010053, 0.5333791967180695, 0.7663316581060059, 0.4298002582667181, 0.6781836812426998, 0.5558391766749284, 0.5224385466734036, 0.6729864884660303, 1.0, 0.8787419089273848, 0.2618449527244832, 0.48633831680799433, 0.2323526860207039, 0.31361832916936394, 0.7098891248983344, 0.3739514690696618, 0.7506238537503395, 0.4379518644116554, 1.0, 0.9554427922043668, 1.0, 0.3855924429638652, 1.0, 0.0, 0.5073659863347824, 0.7826164119688067, 1.0, 0.7890139526519

Calcolo punteggi BLEU: 100%|██████████| 383/383 [00:03<00:00, 111.34it/s]

BLEU scores for Qwen2.5-7B-Instruct-ge (ge): [[1.0, 1.0, 0.0, 0.40737303500287686, 0.0, 0.5451087893190223, 0.4596980088392874, 0.24026767328685925, 0.0, 0.564926870711699, 0.33725973214458643, 0.7075494147561419, 0.8132882808488929, 0.0, 0.5156626918239822, 0.0, 0.46563348805256366, 0.7941386679207173, 0.36191740494054153, 1.0, 0.5037744498395924, 0.6120737901860179, 0.4354294657746997, 0.0, 0.21186050864016664, 1.0, 1.0, 0.23636142259818999, 0.4438233377140732, 0.4408231875586727, 0.47750342648354643, 0.7575151896815032, 0.7490325070687434, 0.321774771113985, 0.6527196653219786, 0.3549481056010052, 0.5096448601550383, 0.5248780147760078, 1.0, 0.9291564424807219, 0.3219583496901924, 0.54413074047686, 0.26567149115019906, 0.14719696960805126, 0.6803749333171202, 0.33266046761138807, 0.7506238537503395, 0.39458812555917666, 1.0, 0.4487432014166947, 0.0, 0.4431197491221154, 1.0, 0.0, 0.517865601584045, 0.41583634222861793, 1.0, 0.6757594721831311, 0.5036586029926082, 0.7016879391277371, 

In [7]:
import pickle

# Save BLEU scores
with open('bleu_scores-exp2.pkl', 'wb') as f:
    pickle.dump(bleu_scores, f)